In [1]:
import re
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder
from collections import Counter
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_validate
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import make_scorer
from sklearn.model_selection import KFold

In [2]:
train = pd.read_table("~/Documents/final project data/train.tsv",sep='\t',index_col=0)

/usr/local/lib/python3.6/site-packages/numpy/lib/arraysetops.py:463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
# train['item_condition_id'] = train['item_condition_id'].astype(np.int8)
train['category_name'] = train['category_name'].fillna('')
train['brand_name'] = train['brand_name'].fillna('')
def rmsle(ytrue,y):
    return np.sqrt(mean_squared_log_error(ytrue,y))

In [6]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=5,random_state=123)
i = 0
def get_val(series,x,default):
    try:
        return series[x]
    except:
        pass
    return default

train['All4'] = ['Cat:'+c+' Brand:'+b+ \
                ' condition:'+str(i) + ' shipping:'+str(s)
                for c,b,i,s in zip(train.category_name,train.brand_name,
                                   train.item_condition_id,train.shipping)]

for train_idx,val_idx in kf.split(train):
    print('Fold {}'.format(i))
    i+=1
    cols = [1,2,3,4,5]
    X_train = train.iloc[train_idx,:]
    X_val = train.iloc[val_idx,:]

    cb_price = X_train.groupby('All4').price.mean()

    mean = train.price.mean()
    
    cb_price_train = np.array([cb_price[x] for x in X_train.All4 ])
    cb_price_val = np.array([get_val(cb_price,x,mean) for x in X_val.All4])
    
    print('RMSLE, train, all feature price: {:0.4}'
          .format(rmsle(X_train.price,cb_price_train)))
    print('RMSLE, test, all feature price: {:0.4}'
          .format(rmsle(X_val.price,cb_price_val)))

Fold 0
RMSLE, train, all feature price: 0.5585
RMSLE, test, all feature price: 0.6014
Fold 1
RMSLE, train, all feature price: 0.5584
RMSLE, test, all feature price: 0.6034
Fold 2
RMSLE, train, all feature price: 0.5582
RMSLE, test, all feature price: 0.6019
Fold 3
RMSLE, train, all feature price: 0.5586
RMSLE, test, all feature price: 0.6008
Fold 4
RMSLE, train, all feature price: 0.5585
RMSLE, test, all feature price: 0.6024
